In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

from pyspark.context import SparkContext
from pyspark.sql.functions import lit, concat_ws, split
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import array, lit
from pyspark.sql import functions as F
from pyspark.sql.functions import array_contains

# se è gia esistente prende lo SparkContext, oppure lo crea
sc = SparkContext.getOrCreate()

# crea una sesione nello spark Context in quanto ci possono essere più session
spark = SparkSession(sc)

sqlContext = SQLContext(sc)

schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("artist", StringType(), True),
    StructField("artist_id", StringType(), True),
    StructField("album", StringType(), True),
    StructField("album_id", StringType(), True),
    StructField("track_number", IntegerType(), True),
    StructField("disk_number", IntegerType(), True),
    StructField("explicit", IntegerType(), True),
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("year", IntegerType(), True),
    StructField("release_date", StringType(), True)
])

df = spark.read.format('csv').options(header=True).load("/content/drive/MyDrive/Colab Notebooks/Data Science/spark/trackid_artist.csv", schema=schema)

#df = df.filter(df.release_date == "0000-00-00")
filtered_artists = df.groupBy("artist").count()

filtered_artists = filtered_artists.filter(filtered_artists["count"] > 200).dropna()

print(filtered_artists.count())
artist_list = filtered_artists.select(['artist']).rdd.flatMap(lambda x: x).collect()
#print(artist_list)

new_list = []

for elem in artist_list:
    if (elem != 'Various Artists' & elem != 'Monstercat Call of the Wild' & elem != '""Missa Sancti Josephi""' & elem != ''):
        new_list.append(elem)

print((new_list))
'''
out = df.filter((df.artist).isin(new_list))
out = out.drop(out.id).drop(out.artist_id).drop(out.album_id).drop(out.explicit).drop(out.year).dropna()
out.createOrReplaceTempView("tracks")

out.show(4000, truncate=False)

#out.printSchema()
#print(out.count())
#out.coalesce(1).write.option("sep",";").options(header=True).csv("./new_tracks/")
'''

In [ ]:
#@title Artist table
import os
import pyspark
  
from pyspark.context import SparkContext
from pyspark.sql.functions import lit, concat_ws, split
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import array, lit
from pyspark.sql import functions as F
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession

# se è gia esistente prende lo SparkContext, oppure lo crea
sc = SparkContext.getOrCreate()
# crea una sesione nello spark Context in quanto ci possono essere più session
spark = SparkSession(sc)

sqlContext = SQLContext(sc)

schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("artist", StringType(), True),
    StructField("artist_id", StringType(), True),
    StructField("album", StringType(), True),
    StructField("album_id", StringType(), True),
    StructField("track_number", IntegerType(), True),
    StructField("disk_number", IntegerType(), True),
    StructField("explicit", IntegerType(), True),
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("year", IntegerType(), True),
    StructField("release_date", StringType(), True)
])

df = spark.read.format('csv').options(header=True).load("/content/drive/MyDrive/Colab Notebooks/Data Science/spark/trackid_artist.csv", schema=schema)

new_list = ['Rage Against The Machine','My Chemical Romance','The Chainsmokers','Aerosmith','Abney Park','Dua Lipa','Tokyo Blade','Ozzy Osbourne','David Bowie','Mariah Carey','Jefferson Airplane','DJ Roc','The Guess Who','Soda Stereo','Ace Frehley','Stevie Nicks','Die Flippers','Linkin Park','The Country Gentlemen','Louis Armstrong','Metallica',
'Electric Light Orchestra','Michael Jackson','Indochine','Zarah Leander','Circus Devils','Green Day','Taylor Dayne','Foo Fighters','Perry Como','Cher','Paul Simon','Sade','The Byrds','The Doors','Lou Reed','Dream Theater','Led Zeppelin','Dire Straits','Jimi Hendrix']

out = df.filter((df.artist).isin(new_list))
out = out.drop(out.artist_id).drop(out.album_id)
#out.show(5000,truncate=False)
out.createOrReplaceTempView("tracks")
out.coalesce(1).write.option("sep",";").options(header=True).csv("/content/drive/MyDrive/Colab Notebooks/spark/toAdd/")
'''


filtered_artists = df.groupBy("artist").count()

filtered_artists = filtered_artists.filter((filtered_artists["count"] < 200) & (filtered_artists["count"] > 100)).dropna()

filtered_artists.orderBy(filtered_artists["count"].desc()).show(5000,truncate=False)

df2 = spark.read.format('csv').options(header=True).option("sep",";").load("/content/drive/MyDrive/Colab Notebooks/spark/artists.csv", schema=schema_artist)

out = df1.join(df2.select('artist', 'origin'), ['artist'])

out.createOrReplaceTempView("tracks")

out.show(5000)
#out.printSchema()
#print(out.count())
#df.coalesce(1).write.option("sep",";").options(header=True).csv("./artists/")
'''

'\n\n\nfiltered_artists = df.groupBy("artist").count()\n\nfiltered_artists = filtered_artists.filter((filtered_artists["count"] < 200) & (filtered_artists["count"] > 100)).dropna()\n\nfiltered_artists.orderBy(filtered_artists["count"].desc()).show(5000,truncate=False)\n\ndf2 = spark.read.format(\'csv\').options(header=True).option("sep",";").load("/content/drive/MyDrive/Colab Notebooks/spark/artists.csv", schema=schema_artist)\n\nout = df1.join(df2.select(\'artist\', \'origin\'), [\'artist\'])\n\nout.createOrReplaceTempView("tracks")\n\nout.show(5000)\n#out.printSchema()\n#print(out.count())\n#df.coalesce(1).write.option("sep",";").options(header=True).csv("./artists/")\n'